In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 4]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...   dam  moa  \
0     prop-1        4    2    4    0    2    5     1   0   2  ...  1.00    0   
1     prop-1        4   12    4    1   19   44    66   2  18  ...  0.00    0   
2     prop-1        4   10    4    0   13   48    45   4  13  ...  1.00    0   
3     prop-1        4    2    3    0    5    7     1   0   5  ...  1.00    0   
4     prop-1        4    8    1    0    4   38     0   3   1  ...  0.50    0   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...   ...  ...   
3017  prop-1        4    7    4    0   18   51    21   0  18  ...  1.00    0   
3018  prop-1        4    3    3    0   27   40     3   0  27  ...  0.00    0   
3019  prop-1        4   10    3    0   19   55    45   0  19  ...  0.20    0   
3020  prop-1        4   30    1    0   58  171   429   9  49  ...  0.00    0   
3021  prop-1        4    4    1    0   10   50     4   0  10  ...  0.75    0   

           mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0     0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1     0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2     0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3     0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4     0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...        ...       ...  ..  ...        ...     ...     ...  ...  
3017  0.647059  0.238095   1    1  55.000000       4  1.5714    0  
3018  0.777778  0.416667   2    2  55.333333       7  2.6667    0  
3019  0.790698  0.333333   2    4  20.800000       4  1.4000    0  
3020  0.000000  0.125490   0    0  75.566667      64  6.0667    0  
3021  0.000000  0.333333   0    0  58.000000      13  3.5000    0  

[3022 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm   lcom3  loc       dam  moa  \
2692    4    1    0   21   45     4   0  21    2  1.0000  175  0.500000    0   
1584    9    1    0   18   80     2   1  18    5  0.8125  623  0.916667    4   
29      0    1    0    0    0     0   0   0    0  2.0000   59  0.000000    0   
2204    2    2    0   21   34     1   0  21    2  1.0000  128  0.000000    0   
2825    2    3    0   15   27     1   0  15    2  2.0000  206  0.000000    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...     ...  ...       ...  ...   
34      2    3    0    5    6     1   0   5    1  2.0000   25  1.000000    0   
1968    2    2    0   11    9     1   0  11    2  2.0000   30  0.000000    0   
2465    1    1    0    6    1     0   2   4    1  2.0000    2  0.000000    0   
2217    4    1    0   17    7     0  13   4    4  0.5000   32  1.000000    0   
1871    1    1    0    3    1     0   3   0    1  2.0000    1  0.000000    0   

           mfa       cam  ic  cbm         amc  max_cc  avg_cc  
2692  0.000000  0.444444   0    0   41.750000       7    2.00  
1584  0.000000  0.562500   0    0   66.888889      22    4.00  
29    0.000000  0.000000   0    0    0.000000       0    0.00  
2204  0.800000  0.666667   0    0   62.000000       1    0.50  
2825  0.909091  0.625000   1    2  102.000000       4    2.00  
...        ...       ...  ..  ...         ...     ...     ...  
34    0.933333  0.750000   1    1   11.000000       4    2.00  
1968  0.800000  0.666667   0    0   14.000000       1    0.50  
2465  0.000000  1.000000   0    0    0.000000       1    1.00  
2217  0.000000  0.437500   0    0    6.500000       1    0.75  
1871  0.000000  1.000000   0    0    0.000000       1    1.00  

[2417 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
1427    3    1    0    6    3     3   3   3    3  2.000000    3  0.0    0   
755     7    3    0    7   22     7   0   7    7  0.833333  174  1.0    0   
2093    6    2    0   20   42     9   1  19    4  0.800000  237  1.0    0   
2224    2    2    0   20   16     1   0  20    2  2.000000   57  0.0    0   
2707    5    1    0    7    5    10   3   4    5  2.000000    5  0.0    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
629     1    3    0    1    2     0   0   1    1  2.000000    4  0.0    0   
2151    7    3    0    6   18    17   1   5    1  1.000000  107  1.0    1   
2408    2    3    0    9   10     1   0   9    2  2.000000   30  0.0    0   
1915    3    4    5   18   25     3   5  13    3  0.500000  265  0.0    0   
2640    2    4    0   18   56     1   0  18    2  2.000000  489  1.0    0   

           mfa       cam  ic  cbm         amc  max_cc  avg_cc  
1427  0.000000  0.666667   0    0    0.000000       1  1.0000  
755   0.866667  0.371429   2    6   23.000000       5  2.2857  
2093  0.900000  0.259259   1    4   38.000000      10  2.6667  
2224  0.800000  0.666667   0    0   27.500000       1  0.5000  
2707  0.000000  0.320000   0    0    0.000000       1  1.0000  
...        ...       ...  ..  ...         ...     ...     ...  
629   1.000000  1.000000   0    0    3.000000       0  0.0000  
2151  0.800000  0.571429   2    4   13.571429       3  1.1429  
2408  0.961538  0.666667   2    2   14.000000       1  0.5000  
1915  0.948718  0.833333   0    0   85.333333      14  5.0000  
2640  0.916667  0.625000   2    2  236.000000       1  0.5000  

[605 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

2692    0
1584    0
29      0
2204    0
2825    0
       ..
34      1
1968    0
2465    0
2217    0
1871    0
Name: bug, Length: 2417, dtype: int64

In [11]:
y_test

1427    0
755     0
2093    0
2224    2
2707    0
       ..
629     1
2151    0
2408    0
1915    0
2640    0
Name: bug, Length: 605, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[ 4.        ,  1.        ,  0.        , ..., 41.75      ,
         7.        ,  2.        ],
       [ 9.        ,  1.        ,  0.        , ..., 66.88888889,
        22.        ,  4.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 4.        ,  1.        ,  0.        , ...,  6.5       ,
         1.        ,  0.75      ],
       [ 1.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [14]:
x_test_np

array([[  3.        ,   1.        ,   0.        , ...,   0.        ,
          1.        ,   1.        ],
       [  7.        ,   3.        ,   0.        , ...,  23.        ,
          5.        ,   2.2857    ],
       [  6.        ,   2.        ,   0.        , ...,  38.        ,
         10.        ,   2.6667    ],
       ...,
       [  2.        ,   3.        ,   0.        , ...,  14.        ,
          1.        ,   0.5       ],
       [  3.        ,   4.        ,   5.        , ...,  85.33333333,
         14.        ,   5.        ],
       [  2.        ,   4.        ,   0.        , ..., 236.        ,
          1.        ,   0.5       ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[ 4.        ,  1.        ,  0.        , ..., 41.75      ,
          7.        ,  2.        ]],

       [[ 9.        ,  1.        ,  0.        , ..., 66.88888889,
         22.        ,  4.        ]],

       [[ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       ...,

       [[ 1.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]],

       [[ 4.        ,  1.        ,  0.        , ...,  6.5       ,
          1.        ,  0.75      ]],

       [[ 1.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]]])

In [18]:
test_y

array([0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv4/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
25/25 [==============================] - 21s 50ms/step - loss: 0.1530 - mse: 0.1530 - mae: 0.1453 - root_mean_squared_error: 0.3912 - mean_squared_logarithmic_error: 0.0499
Epoch 2/100
25/25 [==============================] - 1s 21ms/step - loss: 0.1493 - mse: 0.1493 - mae: 0.1885 - root_mean_squared_error: 0.3864 - mean_squared_logarithmic_error: 0.0502
Epoch 3/100
25/25 [==============================] - 1s 22ms/step - loss: 0.1486 - mse: 0.1486 - mae: 0.1716 - root_mean_squared_error: 0.3855 - mean_squared_logarithmic_error: 0.0486
Epoch 4/100
25/25 [==============================] - 1s 22ms/step - loss: 0.1455 - mse: 0.1455 - mae: 0.1801 - root_mean_squared_error: 0.3814 - mean_squared_logarithmic_error: 0.0480
Epoch 5/100
25/25 [==============================] - 1s 21ms/step - loss: 0.1420 - mse: 0.1420 - mae: 0.1672 - root_mean_squared_error: 0.3768 - mean_squared_logarithmic_error: 0.0463
Epoch 6/100
25/25 [==============================] - 1s 22ms/step - l

25/25 [==============================] - 0s 19ms/step - loss: 0.0809 - mse: 0.0809 - mae: 0.1048 - root_mean_squared_error: 0.2844 - mean_squared_logarithmic_error: 0.0301
Epoch 89/100
25/25 [==============================] - 0s 18ms/step - loss: 0.0838 - mse: 0.0838 - mae: 0.1027 - root_mean_squared_error: 0.2895 - mean_squared_logarithmic_error: 0.0309
Epoch 90/100
25/25 [==============================] - 0s 18ms/step - loss: 0.0827 - mse: 0.0827 - mae: 0.1021 - root_mean_squared_error: 0.2875 - mean_squared_logarithmic_error: 0.0301
Epoch 91/100
25/25 [==============================] - 0s 18ms/step - loss: 0.0806 - mse: 0.0806 - mae: 0.1041 - root_mean_squared_error: 0.2839 - mean_squared_logarithmic_error: 0.0310
Epoch 92/100
25/25 [==============================] - 0s 18ms/step - loss: 0.0844 - mse: 0.0844 - mae: 0.0994 - root_mean_squared_error: 0.2905 - mean_squared_logarithmic_error: 0.0303
Epoch 93/100
25/25 [==============================] - 0s 19ms/step - loss: 0.0841 - mse:

25/25 [==============================] - 1s 22ms/step - loss: 0.0992 - mse: 0.0992 - mae: 0.1056 - root_mean_squared_error: 0.3150 - mean_squared_logarithmic_error: 0.0337
Epoch 74/100
25/25 [==============================] - 1s 22ms/step - loss: 0.0900 - mse: 0.0900 - mae: 0.1077 - root_mean_squared_error: 0.2999 - mean_squared_logarithmic_error: 0.0326
Epoch 75/100
25/25 [==============================] - 1s 22ms/step - loss: 0.0933 - mse: 0.0933 - mae: 0.1053 - root_mean_squared_error: 0.3055 - mean_squared_logarithmic_error: 0.0328
Epoch 76/100
25/25 [==============================] - 1s 21ms/step - loss: 0.0884 - mse: 0.0884 - mae: 0.0983 - root_mean_squared_error: 0.2974 - mean_squared_logarithmic_error: 0.0309
Epoch 77/100
25/25 [==============================] - 1s 21ms/step - loss: 0.0961 - mse: 0.0961 - mae: 0.1006 - root_mean_squared_error: 0.3100 - mean_squared_logarithmic_error: 0.0326
Epoch 78/100
25/25 [==============================] - 0s 19ms/step - loss: 0.0985 - mse:

25/25 [==============================] - 1s 24ms/step - loss: 0.0958 - mse: 0.0958 - mae: 0.1127 - root_mean_squared_error: 0.3095 - mean_squared_logarithmic_error: 0.0337
Epoch 58/100
25/25 [==============================] - 1s 21ms/step - loss: 0.0988 - mse: 0.0988 - mae: 0.1103 - root_mean_squared_error: 0.3144 - mean_squared_logarithmic_error: 0.0338
Epoch 59/100
25/25 [==============================] - 1s 22ms/step - loss: 0.1027 - mse: 0.1027 - mae: 0.1076 - root_mean_squared_error: 0.3205 - mean_squared_logarithmic_error: 0.0351
Epoch 60/100
25/25 [==============================] - 1s 22ms/step - loss: 0.1049 - mse: 0.1049 - mae: 0.1170 - root_mean_squared_error: 0.3238 - mean_squared_logarithmic_error: 0.0369
Epoch 61/100
25/25 [==============================] - 1s 21ms/step - loss: 0.0963 - mse: 0.0963 - mae: 0.1070 - root_mean_squared_error: 0.3103 - mean_squared_logarithmic_error: 0.0334
Epoch 62/100
25/25 [==============================] - 1s 21ms/step - loss: 0.1024 - mse:

25/25 [==============================] - 1s 22ms/step - loss: 0.1082 - mse: 0.1082 - mae: 0.1112 - root_mean_squared_error: 0.3290 - mean_squared_logarithmic_error: 0.0368
Epoch 42/100
25/25 [==============================] - 1s 23ms/step - loss: 0.1028 - mse: 0.1028 - mae: 0.1186 - root_mean_squared_error: 0.3206 - mean_squared_logarithmic_error: 0.0370
Epoch 43/100
25/25 [==============================] - 1s 22ms/step - loss: 0.1061 - mse: 0.1061 - mae: 0.1173 - root_mean_squared_error: 0.3257 - mean_squared_logarithmic_error: 0.0363
Epoch 44/100
25/25 [==============================] - 1s 22ms/step - loss: 0.0994 - mse: 0.0994 - mae: 0.1128 - root_mean_squared_error: 0.3153 - mean_squared_logarithmic_error: 0.0350
Epoch 45/100
25/25 [==============================] - 1s 22ms/step - loss: 0.0938 - mse: 0.0938 - mae: 0.1101 - root_mean_squared_error: 0.3063 - mean_squared_logarithmic_error: 0.0333
Epoch 46/100
25/25 [==============================] - 1s 22ms/step - loss: 0.0927 - mse:

...

25/25 [==============================] - 1s 21ms/step - loss: 0.0968 - mse: 0.0968 - mae: 0.0967 - root_mean_squared_error: 0.3111 - mean_squared_logarithmic_error: 0.0328
Epoch 100/100
19/19 [==============================] - 4s 8ms/step - loss: 0.2601 - mse: 0.2601 - mae: 0.1832 - root_mean_squared_error: 0.5100 - mean_squared_logarithmic_error: 0.0759
score of 5th run is: 0.26009008288383484, 0.26009008288383484, 0.1832202821969986, 0.5099902749061584, 0.07590818405151367
Run no: 6
Epoch 1/100
25/25 [==============================] - 19s 67ms/step - loss: 0.1539 - mse: 0.1539 - mae: 0.1521 - root_mean_squared_error: 0.3923 - mean_squared_logarithmic_error: 0.0507
Epoch 2/100
25/25 [==============================] - 0s 19ms/step - loss: 0.1492 - mse: 0.1492 - mae: 0.1754 - root_mean_squared_error: 0.3863 - mean_squared_logarithmic_error: 0.0491
Epoch 3/100
25/25 [==============================] - 0s 19ms/step - loss: 0.1469 - mse: 0.1469 - mae: 0.1887 - root_mean_squared_error: 0.383

25/25 [==============================] - 0s 17ms/step - loss: 0.0869 - mse: 0.0869 - mae: 0.0980 - root_mean_squared_error: 0.2948 - mean_squared_logarithmic_error: 0.0307
Epoch 85/100
25/25 [==============================] - 0s 18ms/step - loss: 0.0884 - mse: 0.0884 - mae: 0.0969 - root_mean_squared_error: 0.2973 - mean_squared_logarithmic_error: 0.0316
Epoch 86/100
25/25 [==============================] - 0s 17ms/step - loss: 0.0866 - mse: 0.0866 - mae: 0.0978 - root_mean_squared_error: 0.2943 - mean_squared_logarithmic_error: 0.0309
Epoch 87/100
25/25 [==============================] - 0s 17ms/step - loss: 0.0848 - mse: 0.0848 - mae: 0.0994 - root_mean_squared_error: 0.2912 - mean_squared_logarithmic_error: 0.0308
Epoch 88/100
25/25 [==============================] - 0s 17ms/step - loss: 0.0942 - mse: 0.0942 - mae: 0.0971 - root_mean_squared_error: 0.3069 - mean_squared_logarithmic_error: 0.0315
Epoch 89/100
25/25 [==============================] - 0s 17ms/step - loss: 0.0945 - mse:

25/25 [==============================] - 1s 24ms/step - loss: 0.1135 - mse: 0.1135 - mae: 0.1100 - root_mean_squared_error: 0.3368 - mean_squared_logarithmic_error: 0.0395
Epoch 70/100
25/25 [==============================] - 1s 24ms/step - loss: 0.1034 - mse: 0.1034 - mae: 0.1102 - root_mean_squared_error: 0.3215 - mean_squared_logarithmic_error: 0.0356
Epoch 71/100
25/25 [==============================] - 1s 25ms/step - loss: 0.0996 - mse: 0.0996 - mae: 0.1159 - root_mean_squared_error: 0.3156 - mean_squared_logarithmic_error: 0.0345
Epoch 72/100
25/25 [==============================] - 1s 25ms/step - loss: 0.0982 - mse: 0.0982 - mae: 0.1078 - root_mean_squared_error: 0.3133 - mean_squared_logarithmic_error: 0.0331
Epoch 73/100
25/25 [==============================] - 1s 24ms/step - loss: 0.1006 - mse: 0.1006 - mae: 0.1039 - root_mean_squared_error: 0.3172 - mean_squared_logarithmic_error: 0.0341
Epoch 74/100
25/25 [==============================] - 1s 24ms/step - loss: 0.1154 - mse:

25/25 [==============================] - 1s 25ms/step - loss: 0.1038 - mse: 0.1038 - mae: 0.1058 - root_mean_squared_error: 0.3222 - mean_squared_logarithmic_error: 0.0348
Epoch 55/100
25/25 [==============================] - 1s 25ms/step - loss: 0.0995 - mse: 0.0995 - mae: 0.1095 - root_mean_squared_error: 0.3154 - mean_squared_logarithmic_error: 0.0343
Epoch 56/100
25/25 [==============================] - 1s 24ms/step - loss: 0.0992 - mse: 0.0992 - mae: 0.1071 - root_mean_squared_error: 0.3149 - mean_squared_logarithmic_error: 0.0342
Epoch 57/100
25/25 [==============================] - 1s 26ms/step - loss: 0.0988 - mse: 0.0988 - mae: 0.1072 - root_mean_squared_error: 0.3143 - mean_squared_logarithmic_error: 0.0341
Epoch 58/100
25/25 [==============================] - 1s 25ms/step - loss: 0.1007 - mse: 0.1007 - mae: 0.1060 - root_mean_squared_error: 0.3174 - mean_squared_logarithmic_error: 0.0343
Epoch 59/100
25/25 [==============================] - 1s 24ms/step - loss: 0.1026 - mse:

25/25 [==============================] - 1s 25ms/step - loss: 0.1070 - mse: 0.1070 - mae: 0.1195 - root_mean_squared_error: 0.3271 - mean_squared_logarithmic_error: 0.0352
Epoch 40/100
25/25 [==============================] - 1s 24ms/step - loss: 0.1103 - mse: 0.1103 - mae: 0.1225 - root_mean_squared_error: 0.3321 - mean_squared_logarithmic_error: 0.0389
Epoch 41/100
25/25 [==============================] - 1s 23ms/step - loss: 0.1094 - mse: 0.1094 - mae: 0.1144 - root_mean_squared_error: 0.3307 - mean_squared_logarithmic_error: 0.0362
Epoch 42/100
25/25 [==============================] - 1s 23ms/step - loss: 0.1034 - mse: 0.1034 - mae: 0.1166 - root_mean_squared_error: 0.3216 - mean_squared_logarithmic_error: 0.0346
Epoch 43/100
25/25 [==============================] - 1s 25ms/step - loss: 0.1092 - mse: 0.1092 - mae: 0.1229 - root_mean_squared_error: 0.3304 - mean_squared_logarithmic_error: 0.0376
Epoch 44/100
25/25 [==============================] - 1s 24ms/step - loss: 0.1137 - mse:

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv4/'

In [23]:
total_training_time

[77.05906009674072,
 78.32400631904602,
 78.62714648246765,
 74.09267735481262,
 77.25591492652893,
 77.08944749832153,
 77.72872185707092,
 90.70387887954712,
 81.17641878128052,
 78.54313230514526]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 26.35334681669871


In [25]:
total_training_loss_history

[{'loss': [0.15301434695720673,
   0.14929316937923431,
   0.14858420193195343,
   0.14548015594482422,
   0.1420041024684906,
   0.14004728198051453,
   0.13844986259937286,
   0.13478532433509827,
   0.13293427228927612,
   0.13189536333084106,
   0.12892980873584747,
   0.12581922113895416,
   0.12860561907291412,
   0.12956808507442474,
   0.12533658742904663,
   0.1218036487698555,
   0.12076853960752487,
   0.11757706850767136,
   0.11483924090862274,
   0.12086956948041916,
   0.11692710965871811,
   0.11450789868831635,
   0.1125277653336525,
   0.11144500225782394,
   0.10886703431606293,
   0.12104013562202454,
   0.1284824013710022,
   0.11861933767795563,
   0.10705201327800751,
   0.10694846510887146,
   0.10359209775924683,
   0.10294987261295319,
   0.11142487823963165,
   0.1027519702911377,
   0.10149002820253372,
   0.11332836747169495,
   0.112086720764637,
   0.10740189999341965,
   0.11391181498765945,
   0.13501186668872833,
   0.11434389650821686,
   0.1070834174

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.03138683140277863
avg mae of training: 0.03644427508115768


In [27]:
total_scores

[[0.33564117550849915,
  0.33564117550849915,
  0.19872643053531647,
  0.5793454647064209,
  0.08853146433830261],
 [0.3340311348438263,
  0.3340311348438263,
  0.20044130086898804,
  0.5779542922973633,
  0.08447729051113129],
 [0.2753658592700958,
  0.2753658592700958,
  0.21362203359603882,
  0.5247531533241272,
  0.07515355944633484],
 [0.2944103479385376,
  0.2944103479385376,
  0.20279426872730255,
  0.5425959229469299,
  0.07806431502103806],
 [0.3088448643684387,
  0.3088448643684387,
  0.1970924586057663,
  0.5557381510734558,
  0.080832839012146],
 [0.26009008288383484,
  0.26009008288383484,
  0.1832202821969986,
  0.5099902749061584,
  0.07590818405151367],
 [0.3174988925457001,
  0.3174988925457001,
  0.19535580277442932,
  0.5634704232215881,
  0.08528082072734833],
 [0.33747968077659607,
  0.33747968077659607,
  0.1832297444343567,
  0.5809299945831299,
  0.08688335865736008],
 [0.29529520869255066,
  0.29529520869255066,
  0.19031541049480438,
  0.5434107184410095,
  0.

In [1]:
a = [[0.33564117550849915,
  0.33564117550849915,
  0.19872643053531647,
  0.5793454647064209,
  0.08853146433830261],
 [0.3340311348438263,
  0.3340311348438263,
  0.20044130086898804,
  0.5779542922973633,
  0.08447729051113129],
 [0.2753658592700958,
  0.2753658592700958,
  0.21362203359603882,
  0.5247531533241272,
  0.07515355944633484],
 [0.2944103479385376,
  0.2944103479385376,
  0.20279426872730255,
  0.5425959229469299,
  0.07806431502103806],
 [0.3088448643684387,
  0.3088448643684387,
  0.1970924586057663,
  0.5557381510734558,
  0.080832839012146],
 [0.26009008288383484,
  0.26009008288383484,
  0.1832202821969986,
  0.5099902749061584,
  0.07590818405151367],
 [0.3174988925457001,
  0.3174988925457001,
  0.19535580277442932,
  0.5634704232215881,
  0.08528082072734833],
 [0.33747968077659607,
  0.33747968077659607,
  0.1832297444343567,
  0.5809299945831299,
  0.08688335865736008],
 [0.29529520869255066,
  0.29529520869255066,
  0.19031541049480438,
  0.5434107184410095,
  0.07999918609857559],
 [0.2701372802257538,
  0.2701372802257538,
  0.19415344297885895,
  0.5197473168373108,
  0.07551917433738708]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [3]:
test_mae

[0.19872643053531647,
 0.20044130086898804,
 0.21362203359603882,
 0.20279426872730255,
 0.1970924586057663,
 0.1832202821969986,
 0.19535580277442932,
 0.1832297444343567,
 0.19031541049480438,
 0.19415344297885895]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.1009598175684611
avg mae: 0.06529837250709533
avg are: 0.03573920950293541


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.0922560803592205
median testing mse: 0.3020700365304947


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.10219522565603256
median testing mae: 0.1962241306900978


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.10674931108951569


In [35]:
test_ARE

[0.10275482,
 0.10275482,
 0.11404958,
 0.1137741,
 0.10220385,
 0.10688705,
 0.11322314,
 0.11157025,
 0.10661157,
 0.098347105]

In [5]:
a = [0.10275482,
 0.10275482,
 0.11404958,
 0.1137741,
 0.10220385,
 0.10688705,
 0.11322314,
 0.11157025,
 0.10661157,
 0.098347105]
sum(a)/10

0.10721762850000001